# import

In [10]:
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
import ast
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from collections import Counter


user = pd.read_csv('./clean_df.csv', index_col=0)


In [94]:
import json

In [66]:
problem = pd.read_excel('./data/problemList.xlsx')

In [2]:
import joblib


# kmeans + knn

In [11]:
user.info()

<class 'pandas.core.frame.DataFrame'>
Index: 83841 entries, 0 to 84021
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  83841 non-null  object
 1   tier                83841 non-null  int64 
 2   rank                83841 non-null  int64 
 3   rightCnt            83841 non-null  int64 
 4   wrongCnt            83841 non-null  int64 
 5   time out            83841 non-null  int64 
 6   memory exceed       83841 non-null  int64 
 7   print exceed        83841 non-null  int64 
 8   runTime error       83841 non-null  int64 
 9   compile error       83841 non-null  int64 
 10  solvedProblemList   83841 non-null  object
 11  triedNotsolvedList  83841 non-null  object
dtypes: int64(9), object(3)
memory usage: 8.3+ MB


In [12]:
# 필요한 칼럼 가져오기
X = user[['rank', 'rightCnt', 'wrongCnt', 'tier']]

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
kmeans = KMeans(n_clusters=50, random_state=0).fit(X_scaled)
labels = kmeans.labels_

user['new_tier'] = labels

/Users/yunju/miniforge3/envs/tave/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [24]:
# 필요한 칼럼만 선택
x = user[['rank', 'rightCnt', 'wrongCnt', 'tier']]
y = user['new_tier']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# KNN 모델 훈련
knn = KNeighborsClassifier(n_neighbors=20)

knn.fit(X_scaled, y_train)


KNeighborsClassifier(n_neighbors=20)

In [25]:
# KNN 모델 저장
joblib.dump(knn, 'knn_model.pkl')

# StandardScaler 객체 저장
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']

In [26]:
# 저장된 모델 불러오기
loaded_knn = joblib.load('knn_model.pkl')
loaded_scaler = joblib.load('scaler.pkl')


In [37]:
# 훈련 데이터셋 저장 -> 이 데이터를 통해서 조회해야됨
filtered_df = user.loc[X_train.index]
filtered_df.to_csv('./data/train_user.csv')

In [80]:
filtered_df = user.loc[X_train.index][['id', 'tier', 'rank', 'rightCnt', 'wrongCnt', 'solvedProblemList']]
filtered_df.to_csv('./data/train_user.csv')

# ease

In [100]:
class EASE:
    def __init__(self, reg_lambda=500):
        self.reg_lambda = reg_lambda
        self.item_weights = None

    def fit(self, X):
        # Gram Matrix 계산
        G = np.dot(X.T, X) + np.eye(X.shape[1]) * self.reg_lambda

        # Gram Matrix의 역행렬 계산
        P = np.linalg.inv(G)

        # Item Weights 계산
        B = P / -np.diag(P)
        np.fill_diagonal(B, 0)
        self.item_weights = B

    def predict(self, X):
        return np.dot(X, self.item_weights)

In [28]:
test = user[user['id'] == 'baefr'][['rank', 'rightCnt', 'wrongCnt', 'tier']]

In [93]:
selected_data = user[user['id'] == 'baefr'][['rank', 'rightCnt', 'wrongCnt', 'tier', 'solvedProblemList']]

# 데이터를 JSON 형식으로 변환
json_result = selected_data.to_json(orient='records')

print(json_result)

[{"rank":10034,"rightCnt":515,"wrongCnt":235,"tier":15,"solvedProblemList":"['1000', '1001', '1002', '1008', '1010', '1011', '1012', '1018', '1037', '1065', '1085', '1107', '1110', '1149', '1152', '1157', '1158', '1181', '1182', '1193', '1194', '1244', '1260', '1261', '1269', '1292', '1316', '1330', '1339', '1357', '1408', '1427', '1436', '1463', '1476', '1527', '1541', '1546', '1600', '1620', '1629', '1644', '1676', '1697', '1699', '1707', '1712', '1715', '1748', '1753', '1759', '1764', '1912', '1913', '1914', '1929', '1931', '1934', '1977', '1978', '1987', '1992', '2004', '2010', '2023', '2098', '2108', '2164', '2178', '2193', '2206', '2225', '2231', '2239', '2290', '2292', '2309', '2420', '2438', '2439', '2440', '2441', '2442', '2443', '2444', '2445', '2446', '2447', '2455', '2460', '2475', '2480', '2490', '2493', '2501', '2506', '2522', '2523', '2525', '2529', '2531', '2557', '2558', '2562', '2563', '2576', '2577', '2580', '2581', '2583', '2587', '2588', '2592', '2606', '2609', '26

In [97]:
test_event = {
    "httpMethod": "GET",
    "body": json.dumps({
        "rank":10034,
        "rightCnt":515,
        "wrongCnt":235,
        "tier":15,
        "solvedProblemList": ""
    })
}

In [149]:
data = json.loads(test_event['body'])
values = pd.DataFrame([data])
value = values.iloc[:,:-1] # 마지막 칼럼은 맞춘 문제 번호들

In [95]:
def handler(event, context):

# HTTP 메소드에 따른 처리
    if event['httpMethod'] == 'GET':
        # 요청 본문에서 데이터 추출
        data = json.loads(event['body'])
        values = pd.DataFrame([data])
        value = values.iloc[:,:-1] # 마지막 칼럼은 맞춘 문제 번호들
    return value

In [99]:
handler(test_event, None)

,rank,rightCnt,wrongCnt,tier
0,10034,515,235,15


In [112]:
scaled_test = loaded_scaler.transform(handler(test_event, None))

In [113]:
distances, indices = loaded_knn.kneighbors([scaled_test[0]])

In [103]:
filtered_df = pd.read_csv('./data/train_user.csv', index_col=0)

In [132]:
# 가장 유사한 데이터들 데이터 프레임으로 만들기
df = filtered_df.iloc[indices[0]]


In [133]:
df.reset_index(drop=True, inplace=True)

In [134]:

# 푼문제를 리스트로 바꿈 
df['solvedProblemList'] = df['solvedProblemList'].apply(ast.literal_eval)
# 모든 문제의 고유 목록 생성
all_problems = set()
for row in df.itertuples():
    all_problems.update(row.solvedProblemList)
all_problems = sorted(all_problems)

# 이진 상호작용 행렬 생성
interaction_matrix = pd.DataFrame(0, index=df['id'], columns=all_problems)

# 푼문제에 1 부여
for row in df.itertuples():
    interaction_matrix.loc[row.id, row.solvedProblemList] = 1



/var/folders/d6/d15px6nn599bb66tcmrq23380000gn/T/ipykernel_5158/2433354510.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['solvedProblemList'] = df['solvedProblemList'].apply(ast.literal_eval)


In [135]:
interaction_matrix

,1000,1001,1002,10025,10026,1003,10039,1004,1005,1007,...,9654,9655,9657,9663,9694,9933,9934,9935,9946,9996
id,,,,,,,,,,,,,,,,,,,,,
baefr,1,1,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
yjin99,1,1,1,0,0,1,0,1,0,0,...,0,1,0,1,0,0,0,1,0,0
rlawldud335,1,1,0,0,1,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
yee0230,1,1,0,0,1,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
iambestrong,1,1,1,0,1,1,0,0,0,1,...,0,1,0,1,0,0,0,0,0,0
wltnjeon0119,1,1,1,0,1,1,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
ccoco,1,1,1,1,1,1,1,0,0,0,...,1,1,0,0,0,0,0,0,0,0
bhgd010,1,1,1,0,1,1,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
gangukss,1,1,0,0,1,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [136]:
# 모델 초기화 및 학습
model = EASE(reg_lambda=100)
model.fit(interaction_matrix)


In [137]:
# 예측을 위한 데이터 준비 
X_test = interaction_matrix[0:1] # 첫번째 데이터로
# 모델을 사용하여 추천 점수 계산
predicted_scores = model.predict(X_test)

In [138]:
predicted_scores

array([[0.91226557, 0.90917593, 0.7571765 , ..., 0.15902938, 0.00810057,
        0.01670625]])

In [139]:
# 칼럼명(문제 번호) 가져오기
column_names = interaction_matrix.columns

In [140]:
# 예측된 점수와 칼럼명 매핑
score_mapping = {column_names[i]: score for i, score in enumerate(predicted_scores[0])}


In [152]:
solved_problems = values['solvedProblemList'][0]  # 사용자가 풀었던 문제 목록

# 사용자가 풀었던 문제를 제외
filtered_scores = {problem: score for problem, score in score_mapping.items() if problem not in solved_problems}

In [153]:
# 점수에 따라 문제 정렬 (내림차순)
sorted_problems = sorted(filtered_scores.items(), key=lambda x: x[1], reverse=True)

# 상위 15개 문제 선택
top_15_problems = sorted_problems[:15]

# 결과 출력
for problem, score in top_15_problems:
    print(f"Problem: {problem}, Score: {score}")


Problem: 10828, Score: 0.38207561368240983
Problem: 9012, Score: 0.38207561368240966
Problem: 1966, Score: 0.3752640635927007
Problem: 1026, Score: 0.3362021542370566
Problem: 1406, Score: 0.32862450239101304
Problem: 11725, Score: 0.3234354624652293
Problem: 1920, Score: 0.32338624830125967
Problem: 1874, Score: 0.31555953679724114
Problem: 9465, Score: 0.30766119344002246
Problem: 2805, Score: 0.30600495316964993
Problem: 1003, Score: 0.3053121011954745
Problem: 2644, Score: 0.2948434624075636
Problem: 1991, Score: 0.2932871592945306
Problem: 9461, Score: 0.2835946562544642
Problem: 2579, Score: 0.2812075719119002


In [144]:
problem = pd.read_excel('./data/problemList.xlsx')

In [145]:
# 해당 문제 번호 확인해보기
top_15_problem_numbers = [problem[0] for problem in top_15_problems]
top_15_problem_numbers = list(map(int, top_15_problem_numbers))
filtered_problems_df = problem[problem['problemId'].isin(top_15_problem_numbers)]


In [146]:
filtered_problems_df

,problemId,title,acceptedUserCount,level,key,bojTagId
3,1003,피보나치 함수,49732,8,['dp'],[25]
26,1026,보물,32625,7,"['greedy', 'math', 'sorting']","[33, 124, 97]"
402,1406,에디터,20848,9,"['data_structures', 'linked_list', 'stack']","[175, 154, 71]"
868,1874,스택 수열,39615,9,"['data_structures', 'stack']","[175, 71]"
913,1920,수 찾기,48481,7,"['binary_search', 'data_structures', 'sorting']","[12, 175, 97]"
957,1966,프린터 큐,32867,8,"['data_structures', 'implementation', 'queue',...","[175, 102, 72, 141]"
981,1991,트리 순회,26256,10,"['recursion', 'trees']","[62, 120]"
1583,2579,계단 오르기,44345,8,['dp'],[25]
1648,2644,촌수계산,18432,9,"['graphs', 'graph_traversal', 'bfs', 'dfs']","[7, 11, 126, 127]"
1806,2805,나무 자르기,33481,9,"['binary_search', 'parametric_search']","[12, 170]"
